# <center>DSI Project<center>
    


In [435]:
import pandas as pd
import yfinance as yf
import datetime
import matplotlib.pyplot as plt
import numpy as np
import warnings
import datetime
import pyfolio

from datetime import timedelta
from sklearn.linear_model import LinearRegression
warnings.filterwarnings("ignore")

In [436]:
# Read Data
data = pd.read_csv("Japanese_Data.csv")
data.head() 

,Date,Close,DSI
0,4/25/2016,111.08,30.0
1,4/26/2016,111.23,26.0
2,4/27/2016,111.26,31.0
3,4/28/2016,108.55,26.0
4,4/29/2016,106.90,26.0


In [437]:
# Set Index
data.set_index("Date")

,Close,DSI
Date,,
4/25/2016,111.08,30.0
4/26/2016,111.23,26.0
4/27/2016,111.26,31.0
4/28/2016,108.55,26.0
4/29/2016,106.90,26.0
...,...,...
4/19/2021,108.09,NaN
4/20/2021,108.07,NaN
4/21/2021,108.09,NaN


In [438]:
np.sum(pd.isnull(data))

Date      0
Close    58
DSI      23
dtype: int64

In [439]:
# Drop NA
data.dropna(inplace=True)
np.sum(pd.isnull(data))

Date     0
Close    0
DSI      0
dtype: int64

In [440]:
# Plot 
%matplotlib widget
data1 = data.iloc[:,1]
data2 = data.iloc[:,0]

fig, ax1 = plt.subplots(figsize=(9, 4))

color = 'tab:red'
#ax1.set_xlabel('Year-Mon')
ax1.set_ylabel('Daily Sentiment Index', color=color)
ax1.plot(data["DSI"], color=color)
ax1.tick_params(axis='y', labelcolor=color)
ax1.yaxis.set_ticks([0,15,30,45,60,75,90,105,120])

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

color = 'tab:blue'
ax2.set_ylabel("Yen", color=color)  # we already handled the x-label with ax1
ax2.plot(data["Close"], color=color)
ax2.tick_params(axis='y', labelcolor=color)
#ax2.yaxis.set_ticks([0,15,30,45,60,75,90,105,120])

#ax1.xaxis.set_ticks(np.arange(0, len(data2)))
ax1.grid()
fig.tight_layout() # otherwise the right y-label is slightly clipped
#plt.title(Future + " Futures Price")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [441]:
Foreign = data.copy()
Foreign['DSI'] = Foreign['DSI'].astype('int')

Foreign['Date'] = Foreign['Date'].astype('datetime64[ns]')
Foreign.index = Foreign['Date']
Foreign = Foreign[['DSI']]

In [442]:
#Trigger and Exit

# Developing Algorithm to derive time series using assigned values of K and A (Kalman Filter Creation)
# Create Variables so it fits data
Foreign["Kalman_Filter"] = Foreign["DSI"][0]
Foreign["Prediction_Error"] = Foreign["DSI"][0]
Foreign["Filter_Error"] = Foreign["DSI"][0]

# Function created to perform Kalman Filter (This function can be changed to view all the values change)
def Kalman_Filter(k,a):
    for t in range(0,len(Foreign["DSI"])):
        Foreign["Kalman_Filter"][t] = (1-k)*a*Foreign["Kalman_Filter"][t-1] + k*Foreign["DSI"][t]
        Foreign["Prediction_Error"][t] = Foreign["DSI"][t] - a*Foreign["Kalman_Filter"][t-1]
        Foreign["Filter_Error"][t] = Foreign["DSI"][t] - Foreign['Kalman_Filter'][t]

    %matplotlib widget
    plt.figure(figsize=(20,10))
    plt.plot(range(0,len(Foreign["DSI"])), Foreign["DSI"])
    plt.plot(range(0,len(Foreign["DSI"])), Foreign["Kalman_Filter"])
    plt.xlabel("Time")
    plt.ylabel("DSI")
    plt.title("Kalman Filter vs. Actual E")
    
    Filter_Error = np.mean(np.square(Foreign["Filter_Error"]))
    Prediction_Error = np.mean(np.mean(Foreign["Prediction_Error"]))
    return(Prediction_Error)

In [443]:
Kalman_Filter(0.15,0.9999)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

4.032679738562091

In [444]:
Foreign["Close"] = data["Close"].values

In [445]:
Foreign["% Change"] = Foreign["Close"].pct_change().shift(-1)

In [446]:
# Implementing a trade strategy of only going long when presented
warnings.filterwarnings('ignore')

# Creating empty columns to save values
Foreign["Signals"] = 0.

# Identifying trade strategy
Foreign.loc[Foreign["Kalman_Filter"] < Foreign["DSI"], "Signals"] = 1 # Long
Foreign.loc[Foreign["Kalman_Filter"] > Foreign["DSI"], "Signals"] = -1 # Short

for i in range(0,len(Foreign)):
    if Foreign["Signals"][i]==1:
        Foreign["% Change"][i] = Foreign["Close"].pct_change().shift(-1)[i]
        
    elif Foreign["Signals"][i]==0:
        Foreign["% Change"][i] = -Foreign["Close"].pct_change().shift(-1)[i]

In [447]:
Foreign.dropna(inplace = True)
Foreign["Profits"] = Foreign["% Change"] + 1

In [448]:
Foreign.head()

,DSI,Kalman_Filter,Prediction_Error,Filter_Error,Close,% Change,Signals,Profits
Date,,,,,,,,
2016-04-25,30,29,0,1,111.08,0.001350,1.0,1.001350
2016-04-26,26,28,-2,-2,111.23,0.000270,-1.0,1.000270
2016-04-27,31,28,3,3,111.26,-0.024357,1.0,0.975643
2016-04-28,26,27,-1,-1,108.55,-0.015200,-1.0,0.984800
2016-04-29,26,26,0,0,106.90,0.003929,0.0,1.003929


In [449]:
def RtoMonthly(R):
    df = np.cumprod(R)
    df.index = pd.to_datetime(df.index)
    df = df.resample('1M').first()
    df.index = df.index+ timedelta(days=1)
    df = df.shift(-1).pct_change().dropna().iloc[:-1,:]
    df.index = [str(i)[:10] for i in df.index]
    return(df)

def get_bt_graph_data(R):   

    yahoo_data_raw = yf.download( tickers = "SPY ^IRX",
            period = "max",
            interval = "1mo")
    yahoo_data = yahoo_data_raw["Close"]

    # Turn 3 mo Annual yield into monthly yield
    yahoo_data["^IRX"] = ((yahoo_data["^IRX"]/4)/100 + 1) ** (1/3) - 1 

    # SPY % change
    yahoo_data["SPY"] = yahoo_data["SPY"].pct_change() + 1

    # Convert Index types into strings
    yahoo_data = yahoo_data.dropna()
    yahoo_data.index = [str(i)[:10] for i in yahoo_data.index]

    data = pd.concat([RtoMonthly(R)+1,  yahoo_data], axis = 1).dropna()
    data.columns = ["MQE Strategy",  "S&P 500", "^IRX"]
    return(data)

def get_abr2(dff, how = "normal"):
    dff = dff.copy()
    if how == "up":
        dff = dff[dff["S&P 500"] > 0]
    if how == "down":
        dff = dff[dff["S&P 500"] < 0]
        
    x = np.array(dff.iloc[:,1]).reshape(-1, 1)
    y = np.array(dff.iloc[:,0])
    model = LinearRegression().fit(x, y)
    return(model.intercept_, model.coef_[0], model.score(x,y))

def maxdrawdown(returns):
    highest = 1
    low = 1
    drawdown = []
    for p in np.cumprod(returns):
        if p < highest:
            if p < low:
                low = p
        else:
            drawdown.append(1-(low/highest))
            low = p
            highest = p
        
    return(max(drawdown))

def GINI_COEF(returns):
    periods = len(returns)
    LorenzCurve = np.cumsum(returns.sort_values( by = "Returns")-1)
    LorenzCurve = pd.DataFrame({ 'Returns': [0]}).append(LorenzCurve)
    Line = LorenzCurve.copy()
    Line["Returns"] = np.arange(0,1+1/periods, 1/periods)*max(LorenzCurve["Returns"])

    UpArea = 0
    for i in range(1,len(returns)):
        UpArea = UpArea + ((Line.iloc[i,:] - LorenzCurve.iloc[i,:] + Line.iloc[i-1,:] - LorenzCurve.iloc[i-1,:]) / 2)

    if min(LorenzCurve["Returns"]) < 0:
        AllArea = (np.abs(min(LorenzCurve["Returns"])) * periods) + ((max(LorenzCurve["Returns"]) * periods)/2)
    else:
        AllArea = ((max(LorenzCurve["Returns"]) * periods)/2)
    gini = UpArea / AllArea
    return(gini[0])

def bucket_returns(data,quantiles):
    returns = data.copy()
    obs = len(returns)
    bucket = obs//quantiles
    compressd = returns.iloc[range(bucket-1, bucket*(quantiles+1)-1, bucket),:]

    j=0
    for i in range(bucket, bucket*(quantiles+1), bucket):
        compressd.iloc[j//bucket,0] = np.cumsum(returns.iloc[j:i,0])[-1]
        j=i
    compressd.columns = ["Returns"]
    return(compressd)

def eval_table(data, excess_data):
    yay = pd.DataFrame()
    strategies = ["MQE Strategy"]
    for strat in strategies:

        df_go = data[[strat, "S&P 500"]]
        df_solo = data[[strat]]

        excess_df_go = excess_data[[strat, "S&P 500"]]
        excess_df_solo = excess_data[[strat]] 

        i = evaluation(df_go-1, df_solo-1, excess_df_go-1, excess_df_solo-1)
        yay = pd.concat([yay, i], axis = 1)

    yay.columns = strategies
    return(yay)

def evaluation(df_go, df_solo, excess_df_go, excess_df_solo):
    alpha, beta, rsquared = get_abr2(excess_df_go)
    upalpha, upbeta, uprsquared = get_abr2(excess_df_go, how = "up")
    downalpha, downbeta, downrsquared = get_abr2(excess_df_go, how = "down")
    
    alpha = 100 * alpha
    upalpha = 100 * upalpha
    downalpha = 100 * downalpha
    
    correl = df_go.corr().iloc[0,1]
    winning = len(df_solo[df_solo > 0].dropna()) / len(df_solo)
    mon_sd = np.std(df_solo)[0]*100
    ann_sd = mon_sd * np.sqrt(12)
    high = np.max(df_solo)[0] * 100
    low = np.min(df_solo)[0] * 100
    
    df = df_solo.copy()
    df.index = pd.to_datetime(df.index)
    ann_ret = np.cumprod(df+1).resample('1Y').first().pct_change().dropna().mean(axis = 0)[0] * 100

    maxdd = maxdrawdown(df_solo.iloc[:,0]+1)* 100
    
    excess_ann_ret = ((np.cumprod(excess_df_solo+1).iloc[-1,0] **(12/len(excess_df_solo)))-1) * 100
    excess_ann_sd = np.std(excess_df_solo)[0]*100* np.sqrt(12)
    sharpe = (excess_ann_ret - 1) / excess_ann_sd
    gini = GINI_COEF(bucket_returns(df_solo, 10))

    out =pd.DataFrame({"Sharpe": [str(round(sharpe,2))],
    "Average Annual Return": [str(round(ann_ret,3))+ "%"],
    "Market Beta": [str(round(beta,2))],
    "Monthly Alpha": [str(round(alpha,2))+ "%"],
    "Correlation with S&P": [str(round(correl,2))],
    "Max Drawdown": [str(round(-maxdd,2))+ "%"],
    "% of Winning Mo.": [str(round(winning*100,2))+ "%"], 
    "Gini": [str(round(gini,3))],
    "Forecast Accuracy": [np.nan],              
    "Up Alpha": [str(round(upalpha,2))+ "%"],
    "Down Alpha": [str(round(downalpha,2))+ "%"],
    "Up Beta": [str(round(upbeta,2))],
    "Down Beta": [str(round(downbeta,2))],       
    "Highest Monthly Return": [str(round(high,2))],
    "Lowest Monthly Return": [str(round(low,2))],
    "Annualized STD of Returns": [str(round(ann_sd,2))+ "%"]}).transpose()

    
    return(out)

def excess_returns(data):
    data = data
    for i in range(2):
        data.iloc[:,i] = data.iloc[:,i] - data.iloc[:,2]
    return(data.iloc[:,:-1])

In [450]:
R = Foreign[["Profits"]]

data_B = get_bt_graph_data(R)
excess_data_B = excess_returns(data_B)
table = eval_table(data_B, excess_data_B)

data_B.index.names = ['Date']

[*********************100%***********************]  2 of 2 completed


In [451]:
table

,MQE Strategy
Sharpe,0.16
Average Annual Return,0.672%
Market Beta,0.02
Monthly Alpha,0.19%
Correlation with S&P,0.03
Max Drawdown,-6.61%
% of Winning Mo.,52.54%
Gini,0.414
Forecast Accuracy,NaN
Up Alpha,0.89%


In [463]:
%matplotlib widget
plt.plot(Foreign["Profits"].cumprod())
plt.title("Equity Line for Strategy ")
plt.xlabel("Investment Horizon")
plt.ylabel("% of Profit")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, '% of Profit')